In [ ]:
!pip install pyspark

In [ ]:
from pyspark.sql import SparkSession
from pyspark import SparkContext, SQLContext

In [ ]:
spark = SparkSession \
        .builder \
        .master ('local[*]') \
        .appName('Exercício Intro') \
        .getOrCreate()

In [ ]:
df_nomes = spark.read.csv('/content/nomes_aleatorios.txt',inferSchema=True)

df_nomes.show(5)

+----------------+
|             _c0|
+----------------+
|  Frances Bennet|
|   Jamie Russell|
|  Edward Kistler|
|   Sheila Maurer|
|Donald Golightly|
+----------------+
only showing top 5 rows



In [ ]:
df_nomes.printSchema()

root
 |-- _c0: string (nullable = true)



In [ ]:
df_nomes = df_nomes.withColumnRenamed('_c0', 'Nomes')
df_nomes.printSchema()
df_nomes.show(5)

root
 |-- Nomes: string (nullable = true)

+----------------+
|           Nomes|
+----------------+
|  Frances Bennet|
|   Jamie Russell|
|  Edward Kistler|
|   Sheila Maurer|
|Donald Golightly|
+----------------+
only showing top 5 rows



In [ ]:
from pyspark.sql.functions import when, rand

df_nomes = df_nomes.withColumn('Escolaridade',when((rand() <= 0.33), 'Fundamental').when((rand() <= 0.66), 'Médio').otherwise('Superior'))

df_nomes.show(10)

+-----------------+------------+
|            Nomes|Escolaridade|
+-----------------+------------+
|   Frances Bennet|       Médio|
|    Jamie Russell|    Superior|
|   Edward Kistler|       Médio|
|    Sheila Maurer|       Médio|
| Donald Golightly|       Médio|
|       David Gray| Fundamental|
|      Joy Bennett|       Médio|
|      Paul Kriese| Fundamental|
|Berniece Ornellas| Fundamental|
|    Brian Farrell|       Médio|
+-----------------+------------+
only showing top 10 rows



In [ ]:
paises = [
    'Argentina', 'Bolívia', 'Brasil', 'Chile', 'Colômbia',
    'Equador', 'Guiana', 'Guiana Francesa', 'Paraguai', 'Peru',
    'Suriname', 'Uruguai', 'Venezuela'
]
df_nomes = df_nomes.withColumn(
    'Pais',
    when((rand() <= 1/13), paises[0])
    .when((rand() <= 2/13), paises[1])
    .when((rand() <= 3/13), paises[2])
    .when((rand() <= 4/13), paises[3])
    .when((rand() <= 5/13), paises[4])
    .when((rand() <= 6/13), paises[5])
    .when((rand() <= 7/13), paises[6])
    .when((rand() <= 8/13), paises[7])
    .when((rand() <= 9/13), paises[8])
    .when((rand() <= 10/13), paises[9])
    .when((rand() <= 11/13), paises[10])
    .when((rand() <= 12/13), paises[11])
    .otherwise(paises[12])
)

df_nomes.show(10)

+-----------------+------------+---------------+
|            Nomes|Escolaridade|           Pais|
+-----------------+------------+---------------+
|   Frances Bennet|       Médio|       Colômbia|
|    Jamie Russell|    Superior|        Bolívia|
|   Edward Kistler|       Médio|           Peru|
|    Sheila Maurer|       Médio|         Brasil|
| Donald Golightly|       Médio|          Chile|
|       David Gray| Fundamental|Guiana Francesa|
|      Joy Bennett|       Médio|Guiana Francesa|
|      Paul Kriese| Fundamental|          Chile|
|Berniece Ornellas| Fundamental|          Chile|
|    Brian Farrell|       Médio|         Brasil|
+-----------------+------------+---------------+
only showing top 10 rows



In [ ]:
from pyspark.sql.functions import floor

df_nomes = df_nomes.withColumn('AnoNascimento', (floor(rand() * (2010 - 1945 + 1)) + 1945))

df_nomes.show(10)

+-----------------+------------+---------------+-------------+
|            Nomes|Escolaridade|           Pais|AnoNascimento|
+-----------------+------------+---------------+-------------+
|   Frances Bennet|       Médio|       Colômbia|         1973|
|    Jamie Russell|    Superior|        Bolívia|         1978|
|   Edward Kistler|       Médio|           Peru|         1949|
|    Sheila Maurer|       Médio|         Brasil|         1951|
| Donald Golightly|       Médio|          Chile|         1989|
|       David Gray| Fundamental|Guiana Francesa|         1949|
|      Joy Bennett|       Médio|Guiana Francesa|         1962|
|      Paul Kriese| Fundamental|          Chile|         1957|
|Berniece Ornellas| Fundamental|          Chile|         1975|
|    Brian Farrell|       Médio|         Brasil|         1973|
+-----------------+------------+---------------+-------------+
only showing top 10 rows



In [ ]:
df_select = df_nomes.filter((df_nomes.AnoNascimento >= 2001) & (df_nomes.AnoNascimento <= 2100))
df_select.show(10)

+---------------+------------+-------+-------------+
|          Nomes|Escolaridade|   Pais|AnoNascimento|
+---------------+------------+-------+-------------+
| Kara Mcelwaine|       Médio|Bolívia|         2006|
|   Ernest Hulet|       Médio|Bolívia|         2005|
|   Charles Hill| Fundamental|  Chile|         2010|
|Jerry Chynoweth|       Médio|Bolívia|         2004|
|     John Meyer| Fundamental| Guiana|         2008|
|    Lisa Baxley| Fundamental| Guiana|         2008|
|     Ned Tester|    Superior|Bolívia|         2002|
|     April Ward|       Médio|Bolívia|         2007|
| Kenneth Winter|    Superior| Brasil|         2007|
|  Joyce Bennett| Fundamental| Brasil|         2003|
+---------------+------------+-------+-------------+
only showing top 10 rows



In [ ]:
df_nomes.createOrReplaceTempView('pessoas')

spark.sql('select * from pessoas where AnoNascimento >= 2001 and AnoNascimento <= 2100').show(10)

+---------------+------------+-------+-------------+
|          Nomes|Escolaridade|   Pais|AnoNascimento|
+---------------+------------+-------+-------------+
| Kara Mcelwaine|       Médio|Bolívia|         2006|
|   Ernest Hulet|       Médio|Bolívia|         2005|
|   Charles Hill| Fundamental|  Chile|         2010|
|Jerry Chynoweth|       Médio|Bolívia|         2004|
|     John Meyer| Fundamental| Guiana|         2008|
|    Lisa Baxley| Fundamental| Guiana|         2008|
|     Ned Tester|    Superior|Bolívia|         2002|
|     April Ward|       Médio|Bolívia|         2007|
| Kenneth Winter|    Superior| Brasil|         2007|
|  Joyce Bennett| Fundamental| Brasil|         2003|
+---------------+------------+-------+-------------+
only showing top 10 rows



In [ ]:
filtro_millennials = df_nomes.filter((df_nomes.AnoNascimento >= 1980) & (df_nomes.AnoNascimento <= 1994))

quant_millennials = filtro_millennials.count()

print(f'Temos {quant_millennials} pessoas que são da geração Millennial')

Temos 2275372 pessoas que são da geração Millennial


In [ ]:
spark.sql('select count(*) as `Quant. Millennials` from pessoas where AnoNascimento >= 1980 and AnoNascimento <= 1994').show()

+------------------+
|Quant. Millennials|
+------------------+
|           2275372|
+------------------+



In [ ]:
from pyspark.sql.functions import col, count, when

df_geracoes = df_nomes.withColumn(
    'Geracao',
    when((col('AnoNascimento') >= 1944) & (col('AnoNascimento') <= 1964), 'Baby Boomer')
    .when((col('AnoNascimento') >= 1965) & (col('AnoNascimento') <= 1979), 'Geração X')
    .when((col('AnoNascimento') >= 1980) & (col('AnoNascimento') <= 1994), 'Geração Y')
    .when((col('AnoNascimento') >= 1995) & (col('AnoNascimento') <= 2015), 'Geração Z')
)

resultado = (df_geracoes.groupBy('Pais', 'Geracao')
    .agg(count('*').alias('Quantidade'))
    .orderBy('Pais', 'Geracao','Quantidade')
)

resultado.show()

+---------+-----------+----------+
|     Pais|    Geracao|Quantidade|
+---------+-----------+----------+
|Argentina|Baby Boomer|    233493|
|Argentina|  Geração X|    174871|
|Argentina|  Geração Y|    175613|
|Argentina|  Geração Z|    186276|
|  Bolívia|Baby Boomer|    430859|
|  Bolívia|  Geração X|    322667|
|  Bolívia|  Geração Y|    322674|
|  Bolívia|  Geração Z|    343594|
|   Brasil|Baby Boomer|    546430|
|   Brasil|  Geração X|    410211|
|   Brasil|  Geração Y|    410396|
|   Brasil|  Geração Z|    436612|
|    Chile|Baby Boomer|    560431|
|    Chile|  Geração X|    420532|
|    Chile|  Geração Y|    420228|
|    Chile|  Geração Z|    447153|
| Colômbia|Baby Boomer|    486050|
| Colômbia|  Geração X|    363789|
| Colômbia|  Geração Y|    363904|
| Colômbia|  Geração Z|    386974|
+---------+-----------+----------+
only showing top 20 rows

